___
#### **Instalação dos módulos necessários**

In [1]:
%pip install -q nltk
%pip install -q numpy
%pip install -q openai
%pip install -q pandas
%pip install -q scikit-learn
%pip install -q spacy
%pip install -q unidecode

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%%bash
python3 -q -m spacy download pt_core_news_sm

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 40.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


#### **Importação dos módulos necessários**

In [3]:
import nltk 
import openai
import os
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import spacy
import unidecode
nltk.download("stopwords")
nlp = spacy.load("pt_core_news_sm")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/leonardo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


___
#### **Função para pré-processamento**

O pré-processamento é uma etapa fundamental para garantir melhores resultados durante a modelagem do problema. Sendo assim, desenvolvemos a função **preprocessing_text** para aplicar as transformações citadas abaixo no texto.

- Conversão do texto para minúsculo

- Remoção de acentos, emojis, espaços em branco, LINKS, números, pontuações e espaços múltiplos do texto

In [4]:
def preprocessing_text(text: str) -> str:

    # Converte texto para minúsculo
    text: str = text.lower()

    # Remove acentos do texto
    text: str = unidecode.unidecode(text)

    # Remove emojis do texto
    text: str = re.sub(r"[^\w\s,?!]", "", text)
    
    # Remove espaços em branco no texto
    text: str = text.strip()

    # Remove LINKS do texto
    text: str = re.sub(r"http[s]?://\S+", "", text)

    # Remove números do texto
    text: str = re.sub(r"\d+", "", text)
    
    # Remove pontuação do texto
    text: str = re.sub(r"[^\w\s]", "", text)

    # Remove vários espaços em branco do texto
    text: str = re.sub(r"\s{2,}", " ", text)

    return text

#### **Função para remoção de stop words**

A remoção de stops words é outra etapa indispenśavel para resolução de problemas de NLP, uma vez que palavras com pouco valor semântico (artigos, pronomes, preposições, conjunções, interjeições etc) são removidas do texto. Como resultado, há uma grande diminuição no ruído das frases, além de uma grande economia computacional, visto que menos dados serão processados. Para isso, implementamos a função **remove_stop_words** com as seguintes variações:

- NLTK: stopwords do pacote NLTK.

- SPACY: stopwords do pacote SPACY.

- ALL: combinação das stopwords dos pacotes NLTK e SPACY.

In [5]:
def remove_stop_words(text: str, mode: str = "all") -> str:
    
    # Remoção de stop words do pacote NLTK
    if mode == "nltk":
        nltk_stop_words: set = set(nltk.corpus.stopwords.words("portuguese"))
        stop_words: list = list(nltk_stop_words)

    # Remoção de stop words do pacote SPACY
    elif mode == "spacy":
        spacy_stop_words: set = set(nlp.Defaults.stop_words)
        stop_words: list = list(spacy_stop_words)

    # Remoção de stop words combinando os pacotes NLTK e SPACY
    else:
        nltk_stop_words: set = set(nltk.corpus.stopwords.words("portuguese"))
        spacy_stop_words: set = set(nlp.Defaults.stop_words)
        stop_words: list = list(nltk_stop_words.union(spacy_stop_words))

    words: list = text.split()
    
    words_filtered: list = [word for word in words if word not in stop_words]

    text_filtered: str = " ".join(words_filtered)

    return text_filtered

#### **Função para vetorização**

Em termos simples, o processo de vetorização consiste na conversão de textos em números. Dessa forma, os computadores conseguem interpretar os números e gerar respostas satisfatórias (ou não) para solução de um determinado problema. No contexto do projeto, desenvolvemos a função **vectorize** com três variações da técnica de vetorização:

- Count Vectorizer: cada palavra é representada por um número; a cada ocorrência da palavra, o contador do número é incrementado; é um método voltado para frequência e não para importância da palavra dado um contexto.

- TF: mede a frequência de uma palavra em relação ao número total de palavras do documento; por exemplo: no texto - "Eu gosto de assistir animes e eu gosto de ler mangás", a palavra "gosto" teria um peso de 0.4 (2/5), visto que existem 5 palavras (gosto, assistir, animes, ler e mangás).

- TFIDF: é uma técnica que combina o TF (frequência do termo) e o IDF (frequência inversa no documento), ou seja, a técnica tende a valorizar palavras que aparecem unicamente nos documentos, logo, palavras que aparecem com frequência entre documentos tornam-se menos relevantes.


In [6]:
def vectorize(custom_ngram: int, function: str = "count_vectorizer") -> CountVectorizer | TfidfVectorizer:
    
    if function == "count_vectorizer":
        vect: CountVectorizer = CountVectorizer(ngram_range=(1, custom_ngram))

    elif function == "tf":
        vect: TfidfVectorizer = TfidfVectorizer(ngram_range=(1, custom_ngram), use_idf=False)

    else:
        vect: TfidfVectorizer = TfidfVectorizer(ngram_range=(1, custom_ngram), use_idf=True)

    return vect

#### **Função para lemmatização**

A lemmatização é uma técnica utilizada para reduzir as palavras aos seus radicais, ou seja, palavras escritas no plural ou com diferentes conjugações seriam representadas da mesma forma, por exemplo: as palavras "sorrir", "sorriram" e "sorririam" seriam interpretadas como "sorrir". Sendo assim, implementamos a função **lemmatize** com duas variações:

- ALL: onde todas palavras serão lemmatizadas.

- VERBS: onde apenas os verbos serão lemmatizados.

In [7]:
def lemmatize(text : str, mode : str = "all") -> str:
    
    lemmatized_text: list = list()

    document = nlp(text)

    for word in document:
        if mode == "all":
            lemmatized_text.append(word.lemma_)
        else:
            if word.pos_ == "VERB":
                lemmatized_text.append(word.lemma_)
            else:
                lemmatized_text.append(word.text)
    
    return " ".join(lemmatized_text)

#### **Função para treinamento, predição e avaliação**

Para facilitar o processo de treinamento, predição e avaliação dos modelos, desenvolvemos a função **fit_predict_evaluate_models**. Utilizando a função, todas as ações citadas anteriormente são realizadas nos modelos DecisionTreeClassifier, LogisticRegression e Support Vector Machine. Por fim, um dicionário contendo os resultados de avaliação dos modelos são retornados.

In [8]:
def fit_predict_evaluate_models(x_train: pd.DataFrame, y_train: pd.DataFrame, x_test: pd.DataFrame, y_test: pd.DataFrame):
    
    # Ajuste dos modelos
    decision_tree_fit: DecisionTreeClassifier = DecisionTreeClassifier(max_depth=10, min_samples_split=4, min_samples_leaf=4, criterion="gini", random_state=42).fit(x_train, y_train)
    logistic_regression_fit: LogisticRegression = LogisticRegression(C=1.0, solver="lbfgs", max_iter=1000, penalty="l2", random_state=42).fit(x_train, y_train)
    svm_fit: svm = svm.SVC(C=1.0, kernel="rbf", gamma="scale", degree=3, random_state=42).fit(x_train, y_train)

    # Predição dos modelos
    y_pred_decision_tree = decision_tree_fit.predict(x_test)
    y_pred_logistic_regression = logistic_regression_fit.predict(x_test)
    y_pred_svm = svm_fit.predict(x_test)

    # Cálculo de métricas dos modelos
    decision_tree_report = classification_report(y_test, y_pred_decision_tree, output_dict=True)
    logistic_regression_report = classification_report(y_test, y_pred_logistic_regression, output_dict=True)
    svm_report = classification_report(y_test, y_pred_svm, output_dict=True)

    # Resultados
    result: dict = {
        "DECISION_TREE" : decision_tree_report, 
        "LOGISTIC_REGRESSION" : logistic_regression_report,
        "SVM" : svm_report
    }

    return result

#### **Função para comunicação com a API da OpenAI**

Para utilizar os Largue Language Models (LLMs) no desenvolvimento do projeto, desenvolvemos a função **start_client** que recebe por input o modelo, a chave de API e a endereço base do modelo que iremos utilizar.

In [9]:
def start_client(model : str, api_key : str, base_url: str = ""):
    if model in ["gpt-4", "gpt-4o-mini", "gpt-3.5-turbo"]:
        return openai.OpenAI(api_key=api_key)
    else:
        return openai.OpenAI(api_key=api_key, base_url=base_url)

#### **Função para escolha e utilização do LLM**

Após autenticação na API, criamos a função **classify_text** para facilitar o teste de diferentes modelos: GPT-4, GPT-4O-MINI, GPT-3.5-TURBO e SABIA-3. Dessa forma, ganharemos tempo durante o desenvolvimento do projeto.

In [10]:
def classify_text(client: openai, model: str, text: str):
    
    if model == ["gpt-4", "gpt-4o-mini", "gpt-3.5-turbo"]:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role" : "system", "content": "Você é um modelo de classificação de texto no contexto de centrais telefônicas, gostaria que classificasse o texto abaixo entre as categorias: serviços de conta bancária; cartão de crédito ou cartão pré-pago; roubo ou relatório de disputa; hipotecas ou empréstimos; outros. O resultado gostaria que seguisse o template de output: {"'"texto"'": <'texto original do input'>, "'"categoria"'": <'categoria classificada pelo chatgpt'>}."},
                {"role" : "user", "content" : f"Texto: {text}"}
            ],
        )
        return response.choices[0].message.content
    else:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role" : "system", "content": "Você é um modelo de classificação de texto no contexto de centrais telefônicas, gostaria que classificasse o texto abaixo entre as categorias: serviços de conta bancária; cartão de crédito ou cartão pré-pago; roubo ou relatório de disputa; hipotecas ou empréstimos; outros. O resultado gostaria que seguisse o template de output: {"'"texto"'": <'texto original do input'>, "'"categoria"'": <'categoria classificada pelo chatgpt'>}."},
                {"role" : "user", "content" : f"Texto: {text}"}
            ],
        )
        return response.choices[0].message.content

___
#### **Leitura e visualização inicial do conjunto de dados**

In [11]:
dataset: pd.DataFrame = pd.read_csv("datasets/dataset.csv", delimiter=";")

In [12]:
dataset.head(5)

,id_reclamacao,data_abertura,categoria,descricao_reclamacao
0,3229299,2019-05-01T12:00:00-05:00,Hipotecas / Empréstimos,"Bom dia, meu nome é xxxx xxxx e agradeço se vo..."
1,3199379,2019-04-02T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,Atualizei meu cartão xxxx xxxx em xx/xx/2018 e...
2,3233499,2019-05-06T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,O cartão Chase foi relatado em xx/xx/2019. No ...
3,3180294,2019-03-14T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,"Em xx/xx/2018, enquanto tentava reservar um ti..."
4,3224980,2019-04-27T12:00:00-05:00,Serviços de conta bancária,"Meu neto me dê cheque por {$ 1600,00} Eu depos..."


In [13]:
dataset.tail(5)

,id_reclamacao,data_abertura,categoria,descricao_reclamacao
21067,3094545,2018-12-07T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,Depois de ser um cliente de cartão de persegui...
21068,3091984,2018-12-05T12:00:00-05:00,Roubo / Relatório de disputa,"Na quarta -feira, xx/xx/xxxx, liguei para o Ch..."
21069,3133355,2019-01-25T12:00:00-05:00,Roubo / Relatório de disputa,Não estou familiarizado com o XXXX Pay e não e...
21070,3110963,2018-12-27T12:00:00-05:00,Outros,Eu tive crédito impecável por 30 anos. Eu tive...
21071,2001189,2016-07-06T12:00:00-05:00,Outros,"Mais de 10 anos atrás, encerrei minhas contas ..."


In [14]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21072 entries, 0 to 21071
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id_reclamacao         21072 non-null  int64 
 1   data_abertura         21072 non-null  object
 2   categoria             21072 non-null  object
 3   descricao_reclamacao  21072 non-null  object
dtypes: int64(1), object(3)
memory usage: 658.6+ KB


In [15]:
dataset["categoria"].value_counts()

categoria
Serviços de conta bancária             5161
Cartão de crédito / Cartão pré-pago    5006
Roubo / Relatório de disputa           4822
Hipotecas / Empréstimos                3850
Outros                                 2233
Name: count, dtype: int64

___
#### **Experimentos realizados**

**Controle de Performance dos Experimentos**

A variável a seguir foi definida para realizar o controle dos experimentos, ou seja, a cada experimento feito, iremos incrementar o valor de i (i += 1) e, em seguida, apresentar os valores do modelo treinado na seção.

In [16]:
i: int = 0
results: dict = {}

**Funções aplicadas no experimento 00**

- PREPROCESSING
- REMOVE_STOP_WORDS (NLTK)
- LEMMATIZE (ALL)
- COUNT_VECTORIZER (UNIGRAMA)

In [17]:
# Definição dos conjuntos de treino (75%) e teste (25%)
dataset_train, dataset_test = train_test_split(dataset, train_size=0.75, random_state=42)

In [18]:
# Pré-processamento e remoção de stop words do conjunto de treinamento
dataset_train["descricao_reclamacao"] = dataset_train["descricao_reclamacao"].apply(lambda x: preprocessing_text(text=x))
dataset_train["descricao_reclamacao"] = dataset_train["descricao_reclamacao"].apply(lambda x: remove_stop_words(text=x, mode="nltk"))
dataset_train["descricao_reclamacao_lemmatize_all"] = dataset_train["descricao_reclamacao"].apply(lambda x: lemmatize(text=x, mode="all"))

# Vetorização do conjunto de treinamento
vect = vectorize(custom_ngram=1, function="count_vectorizer")
vect.fit(dataset_train["descricao_reclamacao_lemmatize_all"])

x_train = vect.transform(dataset_train["descricao_reclamacao_lemmatize_all"])
y_train = dataset_train["categoria"]

In [19]:
# Pré-processamento e remoção de stop words do conjunto de teste
dataset_test["descricao_reclamacao"] = dataset_test["descricao_reclamacao"].apply(lambda x: preprocessing_text(text=x))
dataset_test["descricao_reclamacao"] = dataset_test["descricao_reclamacao"].apply(lambda x: remove_stop_words(text=x, mode="nltk"))
dataset_test["descricao_reclamacao_lemmatize_all"] = dataset_test["descricao_reclamacao"].apply(lambda x: lemmatize(text=x, mode="all"))

# Vetorização do conjunto de teste
x_test = vect.transform(dataset_test["descricao_reclamacao_lemmatize_all"])
y_test = dataset_test["categoria"]

In [20]:
# Treinamento, predição e avaliação dos modelos
experiment  = fit_predict_evaluate_models(x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test)

# Adiciona os resultados experimentos ao dicionário de resultados
results[f"{str(i).zfill(2)}"] = experiment

# Apresenta os resultados para cada modelo do experimento
print("DECISION TREE:", results[f"{str(i).zfill(2)}"]["DECISION_TREE"]["weighted avg"])
print("LOGISTIC REGRESSION:", results[f"{str(i).zfill(2)}"]["LOGISTIC_REGRESSION"]["weighted avg"])
print("SUPPORT VECTOR MACHINE:", results[f"{str(i).zfill(2)}"]["SVM"]["weighted avg"])

DECISION TREE: {'precision': 0.7274240481843449, 'recall': 0.7249430523917996, 'f1-score': 0.72417155191262, 'support': 5268.0}
LOGISTIC REGRESSION: {'precision': 0.9028120908594506, 'recall': 0.9028094153378892, 'f1-score': 0.9027825477411877, 'support': 5268.0}
SUPPORT VECTOR MACHINE: {'precision': 0.8839674550982918, 'recall': 0.8840167046317388, 'f1-score': 0.8834091588588696, 'support': 5268.0}


**Funções aplicadas no experimento 01**

- PREPROCESSING
- REMOVE_STOP_WORDS (SPACY)
- LEMMATIZE (ALL)
- COUNT_VECTORIZER (UNIGRAMA)

In [21]:
# Definição dos conjuntos de treino (75%) e teste (25%)
dataset_train, dataset_test = train_test_split(dataset, train_size=0.75, random_state=42)

In [22]:
# Pré-processamento e remoção de stop words do conjunto de treinamento
dataset_train["descricao_reclamacao"] = dataset_train["descricao_reclamacao"].apply(lambda x: preprocessing_text(text=x))
dataset_train["descricao_reclamacao"] = dataset_train["descricao_reclamacao"].apply(lambda x: remove_stop_words(text=x, mode="spacy"))
dataset_train["descricao_reclamacao_lemmatize_all"] = dataset_train["descricao_reclamacao"].apply(lambda x: lemmatize(text=x, mode="all"))

# Vetorização do conjunto de treinamento
vect = vectorize(custom_ngram=1, function="count_vectorizer")
vect.fit(dataset_train["descricao_reclamacao_lemmatize_all"])

x_train = vect.transform(dataset_train["descricao_reclamacao_lemmatize_all"])
y_train = dataset_train["categoria"]

In [23]:
# Pré-processamento e remoção de stop words do conjunto de teste
dataset_test["descricao_reclamacao"] = dataset_test["descricao_reclamacao"].apply(lambda x: preprocessing_text(text=x))
dataset_test["descricao_reclamacao"] = dataset_test["descricao_reclamacao"].apply(lambda x: remove_stop_words(text=x, mode="spacy"))
dataset_test["descricao_reclamacao_lemmatize_all"] = dataset_test["descricao_reclamacao"].apply(lambda x: lemmatize(text=x, mode="all"))

# Vetorização do conjunto de teste
x_test = vect.transform(dataset_test["descricao_reclamacao_lemmatize_all"])
y_test = dataset_test["categoria"]

In [24]:
# Treinamento, predição e avaliação dos modelos
experiment  = fit_predict_evaluate_models(x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test)

# Incrementa o contador de experimentos
i += 1

# Adiciona os resultados experimentos ao dicionário de resultados
results[f"{str(i).zfill(2)}"] = experiment

# Apresenta os resultados para cada modelo do experimento
print("DECISION TREE:", results[f"{str(i).zfill(2)}"]["DECISION_TREE"]["weighted avg"])
print("LOGISTIC REGRESSION:", results[f"{str(i).zfill(2)}"]["LOGISTIC_REGRESSION"]["weighted avg"])
print("SUPPORT VECTOR MACHINE:", results[f"{str(i).zfill(2)}"]["SVM"]["weighted avg"])

DECISION TREE: {'precision': 0.731519833974445, 'recall': 0.7317767653758542, 'f1-score': 0.7297681461565174, 'support': 5268.0}
LOGISTIC REGRESSION: {'precision': 0.9001494000798428, 'recall': 0.9001518602885346, 'f1-score': 0.9001464481723435, 'support': 5268.0}
SUPPORT VECTOR MACHINE: {'precision': 0.8815038943769329, 'recall': 0.8817388003037205, 'f1-score': 0.8812424375073433, 'support': 5268.0}


**Funções aplicadas no experimento 02**

- PREPROCESSING
- REMOVE_STOP_WORDS (NLTK + SPACY)
- LEMMATIZE (ALL)
- COUNT_VECTORIZER (UNIGRAMA)

In [25]:
# Definição dos conjuntos de treino (75%) e teste (25%)
dataset_train, dataset_test = train_test_split(dataset, train_size=0.75, random_state=42)

In [26]:
# Pré-processamento e remoção de stop words do conjunto de treinamento
dataset_train["descricao_reclamacao"] = dataset_train["descricao_reclamacao"].apply(lambda x: preprocessing_text(text=x))
dataset_train["descricao_reclamacao"] = dataset_train["descricao_reclamacao"].apply(lambda x: remove_stop_words(text=x, mode="all"))
dataset_train["descricao_reclamacao_lemmatize_all"] = dataset_train["descricao_reclamacao"].apply(lambda x: lemmatize(text=x, mode="all"))

# Vetorização do conjunto de treinamento
vect = vectorize(custom_ngram=1, function="count_vectorizer")
vect.fit(dataset_train["descricao_reclamacao_lemmatize_all"])

x_train = vect.transform(dataset_train["descricao_reclamacao_lemmatize_all"])
y_train = dataset_train["categoria"]

In [27]:
# Pré-processamento e remoção de stop words do conjunto de teste
dataset_test["descricao_reclamacao"] = dataset_test["descricao_reclamacao"].apply(lambda x: preprocessing_text(text=x))
dataset_test["descricao_reclamacao"] = dataset_test["descricao_reclamacao"].apply(lambda x: remove_stop_words(text=x, mode="all"))
dataset_test["descricao_reclamacao_lemmatize_all"] = dataset_test["descricao_reclamacao"].apply(lambda x: lemmatize(text=x, mode="all"))

# Vetorização do conjunto de teste
x_test = vect.transform(dataset_test["descricao_reclamacao_lemmatize_all"])
y_test = dataset_test["categoria"]

In [28]:
# Treinamento, predição e avaliação dos modelos
experiment  = fit_predict_evaluate_models(x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test)

# Incrementa o contador de experimentos
i += 1

# Adiciona os resultados experimentos ao dicionário de resultados
results[f"{str(i).zfill(2)}"] = experiment

# Apresenta os resultados para cada modelo do experimento
print("DECISION TREE:", results[f"{str(i).zfill(2)}"]["DECISION_TREE"]["weighted avg"])
print("LOGISTIC REGRESSION:", results[f"{str(i).zfill(2)}"]["LOGISTIC_REGRESSION"]["weighted avg"])
print("SUPPORT VECTOR MACHINE:", results[f"{str(i).zfill(2)}"]["SVM"]["weighted avg"])

DECISION TREE: {'precision': 0.7287218355287862, 'recall': 0.7291192103264996, 'f1-score': 0.7275672840716497, 'support': 5268.0}
LOGISTIC REGRESSION: {'precision': 0.8989939588750506, 'recall': 0.8990129081245254, 'f1-score': 0.8989977457577429, 'support': 5268.0}
SUPPORT VECTOR MACHINE: {'precision': 0.8814934825232511, 'recall': 0.8817388003037205, 'f1-score': 0.8811965910536949, 'support': 5268.0}


**Funções aplicadas no experimento 03**

- PREPROCESSING
- REMOVE_STOP_WORDS (NLTK)
- LEMMATIZE (ALL)
- COUNT_VECTORIZER (BIGRAMA)

In [29]:
# Definição dos conjuntos de treino (75%) e teste (25%)
dataset_train, dataset_test = train_test_split(dataset, train_size=0.75, random_state=42)

In [30]:
# Pré-processamento e remoção de stop words do conjunto de treinamento
dataset_train["descricao_reclamacao"] = dataset_train["descricao_reclamacao"].apply(lambda x: preprocessing_text(text=x))
dataset_train["descricao_reclamacao"] = dataset_train["descricao_reclamacao"].apply(lambda x: remove_stop_words(text=x, mode="nltk"))
dataset_train["descricao_reclamacao_lemmatize_all"] = dataset_train["descricao_reclamacao"].apply(lambda x: lemmatize(text=x, mode="all"))

# Vetorização do conjunto de treinamento
vect = vectorize(custom_ngram=2, function="count_vectorizer")
vect.fit(dataset_train["descricao_reclamacao_lemmatize_all"])

x_train = vect.transform(dataset_train["descricao_reclamacao_lemmatize_all"])
y_train = dataset_train["categoria"]

In [31]:
# Pré-processamento e remoção de stop words do conjunto de teste
dataset_test["descricao_reclamacao"] = dataset_test["descricao_reclamacao"].apply(lambda x: preprocessing_text(text=x))
dataset_test["descricao_reclamacao"] = dataset_test["descricao_reclamacao"].apply(lambda x: remove_stop_words(text=x, mode="nltk"))
dataset_test["descricao_reclamacao_lemmatize_all"] = dataset_test["descricao_reclamacao"].apply(lambda x: lemmatize(text=x, mode="all"))

# Vetorização do conjunto de teste
x_test = vect.transform(dataset_test["descricao_reclamacao_lemmatize_all"])
y_test = dataset_test["categoria"]

In [32]:
# Treinamento, predição e avaliação dos modelos
experiment  = fit_predict_evaluate_models(x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test)

# Incrementa o contador de experimentos
i += 1

# Adiciona os resultados experimentos ao dicionário de resultados
results[f"{str(i).zfill(2)}"] = experiment

# Apresenta os resultados para cada modelo do experimento
print("DECISION TREE:", results[f"{str(i).zfill(2)}"]["DECISION_TREE"]["weighted avg"])
print("LOGISTIC REGRESSION:", results[f"{str(i).zfill(2)}"]["LOGISTIC_REGRESSION"]["weighted avg"])
print("SUPPORT VECTOR MACHINE:", results[f"{str(i).zfill(2)}"]["SVM"]["weighted avg"])

DECISION TREE: {'precision': 0.7228488258024449, 'recall': 0.7209567198177677, 'f1-score': 0.7199297377542196, 'support': 5268.0}
LOGISTIC REGRESSION: {'precision': 0.9100935779604845, 'recall': 0.9102126044039484, 'f1-score': 0.9101018408208321, 'support': 5268.0}
SUPPORT VECTOR MACHINE: {'precision': 0.8795352870993577, 'recall': 0.8796507213363706, 'f1-score': 0.8791310254528112, 'support': 5268.0}


**Funções aplicadas no experimento 04**

- PREPROCESSING
- REMOVE_STOP_WORDS (SPACY)
- LEMMATIZE (ALL)
- COUNT_VECTORIZER (BIGRAMA)

In [33]:
# Definição dos conjuntos de treino (75%) e teste (25%)
dataset_train, dataset_test = train_test_split(dataset, train_size=0.75, random_state=42)

In [34]:
# Pré-processamento e remoção de stop words do conjunto de treinamento
dataset_train["descricao_reclamacao"] = dataset_train["descricao_reclamacao"].apply(lambda x: preprocessing_text(text=x))
dataset_train["descricao_reclamacao"] = dataset_train["descricao_reclamacao"].apply(lambda x: remove_stop_words(text=x, mode="spacy"))
dataset_train["descricao_reclamacao_lemmatize_all"] = dataset_train["descricao_reclamacao"].apply(lambda x: lemmatize(text=x, mode="all"))

# Vetorização do conjunto de treinamento
vect = vectorize(custom_ngram=2, function="count_vectorizer")
vect.fit(dataset_train["descricao_reclamacao_lemmatize_all"])

x_train = vect.transform(dataset_train["descricao_reclamacao_lemmatize_all"])
y_train = dataset_train["categoria"]

In [35]:
# Pré-processamento e remoção de stop words do conjunto de teste
dataset_test["descricao_reclamacao"] = dataset_test["descricao_reclamacao"].apply(lambda x: preprocessing_text(text=x))
dataset_test["descricao_reclamacao"] = dataset_test["descricao_reclamacao"].apply(lambda x: remove_stop_words(text=x, mode="spacy"))
dataset_test["descricao_reclamacao_lemmatize_all"] = dataset_test["descricao_reclamacao"].apply(lambda x: lemmatize(text=x, mode="all"))

# Vetorização do conjunto de teste
x_test = vect.transform(dataset_test["descricao_reclamacao_lemmatize_all"])
y_test = dataset_test["categoria"]

In [36]:
# Treinamento, predição e avaliação dos modelos
experiment  = fit_predict_evaluate_models(x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test)

# Incrementa o contador de experimentos
i += 1

# Adiciona os resultados experimentos ao dicionário de resultados
results[f"{str(i).zfill(2)}"] = experiment

# Apresenta os resultados para cada modelo do experimento
print("DECISION TREE:", results[f"{str(i).zfill(2)}"]["DECISION_TREE"]["weighted avg"])
print("LOGISTIC REGRESSION:", results[f"{str(i).zfill(2)}"]["LOGISTIC_REGRESSION"]["weighted avg"])
print("SUPPORT VECTOR MACHINE:", results[f"{str(i).zfill(2)}"]["SVM"]["weighted avg"])

DECISION TREE: {'precision': 0.725878784040775, 'recall': 0.7262718299164769, 'f1-score': 0.7243966262052767, 'support': 5268.0}
LOGISTIC REGRESSION: {'precision': 0.912356970270501, 'recall': 0.9124905087319666, 'f1-score': 0.9123818089139148, 'support': 5268.0}
SUPPORT VECTOR MACHINE: {'precision': 0.880231180609849, 'recall': 0.8804100227790432, 'f1-score': 0.8799272235517006, 'support': 5268.0}


**Funções aplicadas no experimento 05**

- PREPROCESSING
- REMOVE_STOP_WORDS (NLTK + SPACY)
- LEMMATIZE (ALL)
- COUNT_VECTORIZER (BIGRAMA)

In [37]:
# Definição dos conjuntos de treino (75%) e teste (25%)
dataset_train, dataset_test = train_test_split(dataset, train_size=0.75, random_state=42)

In [38]:
# Pré-processamento e remoção de stop words do conjunto de treinamento
dataset_train["descricao_reclamacao"] = dataset_train["descricao_reclamacao"].apply(lambda x: preprocessing_text(text=x))
dataset_train["descricao_reclamacao"] = dataset_train["descricao_reclamacao"].apply(lambda x: remove_stop_words(text=x, mode="all"))
dataset_train["descricao_reclamacao_lemmatize_all"] = dataset_train["descricao_reclamacao"].apply(lambda x: lemmatize(text=x, mode="all"))

# Vetorização do conjunto de treinamento
vect = vectorize(custom_ngram=2, function="count_vectorizer")
vect.fit(dataset_train["descricao_reclamacao_lemmatize_all"])

x_train = vect.transform(dataset_train["descricao_reclamacao_lemmatize_all"])
y_train = dataset_train["categoria"]

In [39]:
# Pré-processamento e remoção de stop words do conjunto de teste
dataset_test["descricao_reclamacao"] = dataset_test["descricao_reclamacao"].apply(lambda x: preprocessing_text(text=x))
dataset_test["descricao_reclamacao"] = dataset_test["descricao_reclamacao"].apply(lambda x: remove_stop_words(text=x, mode="all"))
dataset_test["descricao_reclamacao_lemmatize_all"] = dataset_test["descricao_reclamacao"].apply(lambda x: lemmatize(text=x, mode="all"))

# Vetorização do conjunto de teste
x_test = vect.transform(dataset_test["descricao_reclamacao_lemmatize_all"])
y_test = dataset_test["categoria"]

In [40]:
# Treinamento, predição e avaliação dos modelos
experiment  = fit_predict_evaluate_models(x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test)

# Incrementa o contador de experimentos
i += 1

# Adiciona os resultados experimentos ao dicionário de resultados
results[f"{str(i).zfill(2)}"] = experiment

# Apresenta os resultados para cada modelo do experimento
print("DECISION TREE:", results[f"{str(i).zfill(2)}"]["DECISION_TREE"]["weighted avg"])
print("LOGISTIC REGRESSION:", results[f"{str(i).zfill(2)}"]["LOGISTIC_REGRESSION"]["weighted avg"])
print("SUPPORT VECTOR MACHINE:", results[f"{str(i).zfill(2)}"]["SVM"]["weighted avg"])

DECISION TREE: {'precision': 0.7254063369933709, 'recall': 0.7258921791951405, 'f1-score': 0.7243699905974461, 'support': 5268.0}
LOGISTIC REGRESSION: {'precision': 0.9125890380960586, 'recall': 0.9126803340926348, 'f1-score': 0.9126079504151651, 'support': 5268.0}
SUPPORT VECTOR MACHINE: {'precision': 0.8814614632056513, 'recall': 0.8815489749430524, 'f1-score': 0.8810646208784726, 'support': 5268.0}


**Funções aplicadas no experimento 06**

- PREPROCESSING
- REMOVE_STOP_WORDS (NLTK)
- LEMMATIZE (ALL)
- TF (UNIGRAMA)

In [41]:
# Definição dos conjuntos de treino (75%) e teste (25%)
dataset_train, dataset_test = train_test_split(dataset, train_size=0.75, random_state=42)

In [42]:
# Pré-processamento e remoção de stop words do conjunto de treinamento
dataset_train["descricao_reclamacao"] = dataset_train["descricao_reclamacao"].apply(lambda x: preprocessing_text(text=x))
dataset_train["descricao_reclamacao"] = dataset_train["descricao_reclamacao"].apply(lambda x: remove_stop_words(text=x, mode="nltk"))
dataset_train["descricao_reclamacao_lemmatize_all"] = dataset_train["descricao_reclamacao"].apply(lambda x: lemmatize(text=x, mode="all"))

# Vetorização do conjunto de treinamento
vect = vectorize(custom_ngram=1, function="tf")
vect.fit(dataset_train["descricao_reclamacao_lemmatize_all"])

x_train = vect.transform(dataset_train["descricao_reclamacao_lemmatize_all"])
y_train = dataset_train["categoria"]

In [43]:
# Pré-processamento e remoção de stop words do conjunto de teste
dataset_test["descricao_reclamacao"] = dataset_test["descricao_reclamacao"].apply(lambda x: preprocessing_text(text=x))
dataset_test["descricao_reclamacao"] = dataset_test["descricao_reclamacao"].apply(lambda x: remove_stop_words(text=x, mode="nltk"))
dataset_test["descricao_reclamacao_lemmatize_all"] = dataset_test["descricao_reclamacao"].apply(lambda x: lemmatize(text=x, mode="all"))

# Vetorização do conjunto de teste
x_test = vect.transform(dataset_test["descricao_reclamacao_lemmatize_all"])
y_test = dataset_test["categoria"]

In [44]:
# Treinamento, predição e avaliação dos modelos
experiment  = fit_predict_evaluate_models(x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test)

# Incrementa o contador de experimentos
i += 1

# Adiciona os resultados experimentos ao dicionário de resultados
results[f"{str(i).zfill(2)}"] = experiment

# Apresenta os resultados para cada modelo do experimento
print("DECISION TREE:", results[f"{str(i).zfill(2)}"]["DECISION_TREE"]["weighted avg"])
print("LOGISTIC REGRESSION:", results[f"{str(i).zfill(2)}"]["LOGISTIC_REGRESSION"]["weighted avg"])
print("SUPPORT VECTOR MACHINE:", results[f"{str(i).zfill(2)}"]["SVM"]["weighted avg"])

DECISION TREE: {'precision': 0.7590274565853553, 'recall': 0.7450645406226272, 'f1-score': 0.7482082107210463, 'support': 5268.0}
LOGISTIC REGRESSION: {'precision': 0.9077315454988181, 'recall': 0.907744874715262, 'f1-score': 0.9076336481904554, 'support': 5268.0}
SUPPORT VECTOR MACHINE: {'precision': 0.9007070072303339, 'recall': 0.9007213363705391, 'f1-score': 0.9006184889250168, 'support': 5268.0}


**Funções aplicadas no experimento 07**

- PREPROCESSING
- REMOVE_STOP_WORDS (SPACY)
- LEMMATIZE (ALL)
- TF (UNIGRAMA)

In [45]:
# Definição dos conjuntos de treino (75%) e teste (25%)
dataset_train, dataset_test = train_test_split(dataset, train_size=0.75, random_state=42)

In [46]:
# Pré-processamento e remoção de stop words do conjunto de treinamento
dataset_train["descricao_reclamacao"] = dataset_train["descricao_reclamacao"].apply(lambda x: preprocessing_text(text=x))
dataset_train["descricao_reclamacao"] = dataset_train["descricao_reclamacao"].apply(lambda x: remove_stop_words(text=x, mode="spacy"))
dataset_train["descricao_reclamacao_lemmatize_all"] = dataset_train["descricao_reclamacao"].apply(lambda x: lemmatize(text=x, mode="all"))

# Vetorização do conjunto de treinamento
vect = vectorize(custom_ngram=1, function="tf")
vect.fit(dataset_train["descricao_reclamacao_lemmatize_all"])

x_train = vect.transform(dataset_train["descricao_reclamacao_lemmatize_all"])
y_train = dataset_train["categoria"]

In [47]:
# Pré-processamento e remoção de stop words do conjunto de teste
dataset_test["descricao_reclamacao"] = dataset_test["descricao_reclamacao"].apply(lambda x: preprocessing_text(text=x))
dataset_test["descricao_reclamacao"] = dataset_test["descricao_reclamacao"].apply(lambda x: remove_stop_words(text=x, mode="spacy"))
dataset_test["descricao_reclamacao_lemmatize_all"] = dataset_test["descricao_reclamacao"].apply(lambda x: lemmatize(text=x, mode="all"))

# Vetorização do conjunto de teste
x_test = vect.transform(dataset_test["descricao_reclamacao_lemmatize_all"])
y_test = dataset_test["categoria"]

In [48]:
# Treinamento, predição e avaliação dos modelos
experiment  = fit_predict_evaluate_models(x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test)

# Incrementa o contador de experimentos
i += 1

# Adiciona os resultados experimentos ao dicionário de resultados
results[f"{str(i).zfill(2)}"] = experiment

# Apresenta os resultados para cada modelo do experimento
print("DECISION TREE:", results[f"{str(i).zfill(2)}"]["DECISION_TREE"]["weighted avg"])
print("LOGISTIC REGRESSION:", results[f"{str(i).zfill(2)}"]["LOGISTIC_REGRESSION"]["weighted avg"])
print("SUPPORT VECTOR MACHINE:", results[f"{str(i).zfill(2)}"]["SVM"]["weighted avg"])

DECISION TREE: {'precision': 0.7483573957813439, 'recall': 0.7374715261958997, 'f1-score': 0.7400768862841659, 'support': 5268.0}
LOGISTIC REGRESSION: {'precision': 0.9082737496679834, 'recall': 0.9083143507972665, 'f1-score': 0.9082281880330897, 'support': 5268.0}
SUPPORT VECTOR MACHINE: {'precision': 0.9023986205478282, 'recall': 0.9024297646165528, 'f1-score': 0.9023363145474079, 'support': 5268.0}


**Funções aplicadas no experimento 08**

- PREPROCESSING
- REMOVE_STOP_WORDS (NLTK + SPACY)
- LEMMATIZE (ALL)
- TF (UNIGRAMA)

In [49]:
# Definição dos conjuntos de treino (75%) e teste (25%)
dataset_train, dataset_test = train_test_split(dataset, train_size=0.75, random_state=42)

In [50]:
# Pré-processamento e remoção de stop words do conjunto de treinamento
dataset_train["descricao_reclamacao"] = dataset_train["descricao_reclamacao"].apply(lambda x: preprocessing_text(text=x))
dataset_train["descricao_reclamacao"] = dataset_train["descricao_reclamacao"].apply(lambda x: remove_stop_words(text=x, mode="all"))
dataset_train["descricao_reclamacao_lemmatize_all"] = dataset_train["descricao_reclamacao"].apply(lambda x: lemmatize(text=x, mode="all"))

# Vetorização do conjunto de treinamento
vect = vectorize(custom_ngram=1, function="tf")
vect.fit(dataset_train["descricao_reclamacao_lemmatize_all"])

x_train = vect.transform(dataset_train["descricao_reclamacao_lemmatize_all"])
y_train = dataset_train["categoria"]

In [51]:
# Pré-processamento e remoção de stop words do conjunto de teste
dataset_test["descricao_reclamacao"] = dataset_test["descricao_reclamacao"].apply(lambda x: preprocessing_text(text=x))
dataset_test["descricao_reclamacao"] = dataset_test["descricao_reclamacao"].apply(lambda x: remove_stop_words(text=x, mode="all"))
dataset_test["descricao_reclamacao_lemmatize_all"] = dataset_test["descricao_reclamacao"].apply(lambda x: lemmatize(text=x, mode="all"))

# Vetorização do conjunto de teste
x_test = vect.transform(dataset_test["descricao_reclamacao_lemmatize_all"])
y_test = dataset_test["categoria"]

In [52]:
# Treinamento, predição e avaliação dos modelos
experiment  = fit_predict_evaluate_models(x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test)

# Incrementa o contador de experimentos
i += 1

# Adiciona os resultados experimentos ao dicionário de resultados
results[f"{str(i).zfill(2)}"] = experiment

# Apresenta os resultados para cada modelo do experimento
print("DECISION TREE:", results[f"{str(i).zfill(2)}"]["DECISION_TREE"]["weighted avg"])
print("LOGISTIC REGRESSION:", results[f"{str(i).zfill(2)}"]["LOGISTIC_REGRESSION"]["weighted avg"])
print("SUPPORT VECTOR MACHINE:", results[f"{str(i).zfill(2)}"]["SVM"]["weighted avg"])

DECISION TREE: {'precision': 0.756240463779386, 'recall': 0.7439255884586181, 'f1-score': 0.7465846023691456, 'support': 5268.0}
LOGISTIC REGRESSION: {'precision': 0.908849115843669, 'recall': 0.908883826879271, 'f1-score': 0.9087923646224497, 'support': 5268.0}
SUPPORT VECTOR MACHINE: {'precision': 0.9027973410789459, 'recall': 0.9028094153378892, 'f1-score': 0.9027054879826875, 'support': 5268.0}


**Funções aplicadas no experimento 09**

- PREPROCESSING
- REMOVE_STOP_WORDS (NLTK)
- LEMMATIZE (ALL)
- TF (BIGRAMA)

In [53]:
# Definição dos conjuntos de treino (75%) e teste (25%)
dataset_train, dataset_test = train_test_split(dataset, train_size=0.75, random_state=42)

In [54]:
# Pré-processamento e remoção de stop words do conjunto de treinamento
dataset_train["descricao_reclamacao"] = dataset_train["descricao_reclamacao"].apply(lambda x: preprocessing_text(text=x))
dataset_train["descricao_reclamacao"] = dataset_train["descricao_reclamacao"].apply(lambda x: remove_stop_words(text=x, mode="nltk"))
dataset_train["descricao_reclamacao_lemmatize_all"] = dataset_train["descricao_reclamacao"].apply(lambda x: lemmatize(text=x, mode="all"))

# Vetorização do conjunto de treinamento
vect = vectorize(custom_ngram=2, function="tf")
vect.fit(dataset_train["descricao_reclamacao_lemmatize_all"])

x_train = vect.transform(dataset_train["descricao_reclamacao_lemmatize_all"])
y_train = dataset_train["categoria"]

In [55]:
# Pré-processamento e remoção de stop words do conjunto de teste
dataset_test["descricao_reclamacao"] = dataset_test["descricao_reclamacao"].apply(lambda x: preprocessing_text(text=x))
dataset_test["descricao_reclamacao"] = dataset_test["descricao_reclamacao"].apply(lambda x: remove_stop_words(text=x, mode="nltk"))
dataset_test["descricao_reclamacao_lemmatize_all"] = dataset_test["descricao_reclamacao"].apply(lambda x: lemmatize(text=x, mode="all"))

# Vetorização do conjunto de teste
x_test = vect.transform(dataset_test["descricao_reclamacao_lemmatize_all"])
y_test = dataset_test["categoria"]

In [56]:
# Treinamento, predição e avaliação dos modelos
experiment  = fit_predict_evaluate_models(x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test)

# Incrementa o contador de experimentos
i += 1

# Adiciona os resultados experimentos ao dicionário de resultados
results[f"{str(i).zfill(2)}"] = experiment

# Apresenta os resultados para cada modelo do experimento
print("DECISION TREE:", results[f"{str(i).zfill(2)}"]["DECISION_TREE"]["weighted avg"])
print("LOGISTIC REGRESSION:", results[f"{str(i).zfill(2)}"]["LOGISTIC_REGRESSION"]["weighted avg"])
print("SUPPORT VECTOR MACHINE:", results[f"{str(i).zfill(2)}"]["SVM"]["weighted avg"])

DECISION TREE: {'precision': 0.748786843797808, 'recall': 0.7477220956719818, 'f1-score': 0.7465592866114322, 'support': 5268.0}
LOGISTIC REGRESSION: {'precision': 0.905844691774582, 'recall': 0.90584662110858, 'f1-score': 0.9057257034864319, 'support': 5268.0}
SUPPORT VECTOR MACHINE: {'precision': 0.8997470330696032, 'recall': 0.8997722095671982, 'f1-score': 0.8996140087327708, 'support': 5268.0}


**Funções aplicadas no experimento 10**

- PREPROCESSING
- REMOVE_STOP_WORDS (SPACY)
- LEMMATIZE (ALL)
- TF (BIGRAMA)

In [57]:
# Definição dos conjuntos de treino (75%) e teste (25%)
dataset_train, dataset_test = train_test_split(dataset, train_size=0.75, random_state=42)

In [58]:
# Pré-processamento e remoção de stop words do conjunto de treinamento
dataset_train["descricao_reclamacao"] = dataset_train["descricao_reclamacao"].apply(lambda x: preprocessing_text(text=x))
dataset_train["descricao_reclamacao"] = dataset_train["descricao_reclamacao"].apply(lambda x: remove_stop_words(text=x, mode="spacy"))
dataset_train["descricao_reclamacao_lemmatize_all"] = dataset_train["descricao_reclamacao"].apply(lambda x: lemmatize(text=x, mode="all"))

# Vetorização do conjunto de treinamento
vect = vectorize(custom_ngram=2, function="tf")
vect.fit(dataset_train["descricao_reclamacao_lemmatize_all"])

x_train = vect.transform(dataset_train["descricao_reclamacao_lemmatize_all"])
y_train = dataset_train["categoria"]

In [59]:
# Pré-processamento e remoção de stop words do conjunto de teste
dataset_test["descricao_reclamacao"] = dataset_test["descricao_reclamacao"].apply(lambda x: preprocessing_text(text=x))
dataset_test["descricao_reclamacao"] = dataset_test["descricao_reclamacao"].apply(lambda x: remove_stop_words(text=x, mode="spacy"))
dataset_test["descricao_reclamacao_lemmatize_all"] = dataset_test["descricao_reclamacao"].apply(lambda x: lemmatize(text=x, mode="all"))

# Vetorização do conjunto de teste
x_test = vect.transform(dataset_test["descricao_reclamacao_lemmatize_all"])
y_test = dataset_test["categoria"]

In [60]:
# Treinamento, predição e avaliação dos modelos
experiment  = fit_predict_evaluate_models(x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test)

# Incrementa o contador de experimentos
i += 1

# Adiciona os resultados experimentos ao dicionário de resultados
results[f"{str(i).zfill(2)}"] = experiment

# Apresenta os resultados para cada modelo do experimento
print("DECISION TREE:", results[f"{str(i).zfill(2)}"]["DECISION_TREE"]["weighted avg"])
print("LOGISTIC REGRESSION:", results[f"{str(i).zfill(2)}"]["LOGISTIC_REGRESSION"]["weighted avg"])
print("SUPPORT VECTOR MACHINE:", results[f"{str(i).zfill(2)}"]["SVM"]["weighted avg"])

DECISION TREE: {'precision': 0.7481871730493928, 'recall': 0.7425968109339408, 'f1-score': 0.7385245136427945, 'support': 5268.0}
LOGISTIC REGRESSION: {'precision': 0.9057893876354692, 'recall': 0.90584662110858, 'f1-score': 0.9057423810309457, 'support': 5268.0}
SUPPORT VECTOR MACHINE: {'precision': 0.8997333134269375, 'recall': 0.8997722095671982, 'f1-score': 0.8996281233860111, 'support': 5268.0}


**Funções aplicadas no experimento 11**

- PREPROCESSING
- REMOVE_STOP_WORDS (NLTK + SPACY)
- LEMMATIZE (ALL)
- TF (BIGRAMA)

In [61]:
# Definição dos conjuntos de treino (75%) e teste (25%)
dataset_train, dataset_test = train_test_split(dataset, train_size=0.75, random_state=42)

In [62]:
# Pré-processamento e remoção de stop words do conjunto de treinamento
dataset_train["descricao_reclamacao"] = dataset_train["descricao_reclamacao"].apply(lambda x: preprocessing_text(text=x))
dataset_train["descricao_reclamacao"] = dataset_train["descricao_reclamacao"].apply(lambda x: remove_stop_words(text=x, mode="all"))
dataset_train["descricao_reclamacao_lemmatize_all"] = dataset_train["descricao_reclamacao"].apply(lambda x: lemmatize(text=x, mode="all"))

# Vetorização do conjunto de treinamento
vect = vectorize(custom_ngram=2, function="tf")
vect.fit(dataset_train["descricao_reclamacao_lemmatize_all"])

x_train = vect.transform(dataset_train["descricao_reclamacao_lemmatize_all"])
y_train = dataset_train["categoria"]

In [63]:
# Pré-processamento e remoção de stop words do conjunto de teste
dataset_test["descricao_reclamacao"] = dataset_test["descricao_reclamacao"].apply(lambda x: preprocessing_text(text=x))
dataset_test["descricao_reclamacao"] = dataset_test["descricao_reclamacao"].apply(lambda x: remove_stop_words(text=x, mode="all"))
dataset_test["descricao_reclamacao_lemmatize_all"] = dataset_test["descricao_reclamacao"].apply(lambda x: lemmatize(text=x, mode="all"))

# Vetorização do conjunto de teste
x_test = vect.transform(dataset_test["descricao_reclamacao_lemmatize_all"])
y_test = dataset_test["categoria"]

In [64]:
# Treinamento, predição e avaliação dos modelos
experiment  = fit_predict_evaluate_models(x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test)

# Incrementa o contador de experimentos
i += 1

# Adiciona os resultados experimentos ao dicionário de resultados
results[f"{str(i).zfill(2)}"] = experiment

# Apresenta os resultados para cada modelo do experimento
print("DECISION TREE:", results[f"{str(i).zfill(2)}"]["DECISION_TREE"]["weighted avg"])
print("LOGISTIC REGRESSION:", results[f"{str(i).zfill(2)}"]["LOGISTIC_REGRESSION"]["weighted avg"])
print("SUPPORT VECTOR MACHINE:", results[f"{str(i).zfill(2)}"]["SVM"]["weighted avg"])

DECISION TREE: {'precision': 0.7584599100268778, 'recall': 0.7471526195899773, 'f1-score': 0.7494943367732017, 'support': 5268.0}
LOGISTIC REGRESSION: {'precision': 0.9063750998630971, 'recall': 0.9064160971905847, 'f1-score': 0.9063047041515369, 'support': 5268.0}
SUPPORT VECTOR MACHINE: {'precision': 0.8993636215452142, 'recall': 0.8993925588458618, 'f1-score': 0.8992392170677924, 'support': 5268.0}


**Funções aplicadas no experimento 12**

- PREPROCESSING
- REMOVE_STOP_WORDS (NLTK)
- LEMMATIZE (ALL)
- TFIDF (UNIGRAMA)

In [65]:
# Definição dos conjuntos de treino (75%) e teste (25%)
dataset_train, dataset_test = train_test_split(dataset, train_size=0.75, random_state=42)

In [66]:
# Pré-processamento e remoção de stop words do conjunto de treinamento
dataset_train["descricao_reclamacao"] = dataset_train["descricao_reclamacao"].apply(lambda x: preprocessing_text(text=x))
dataset_train["descricao_reclamacao"] = dataset_train["descricao_reclamacao"].apply(lambda x: remove_stop_words(text=x, mode="nltk"))
dataset_train["descricao_reclamacao_lemmatize_all"] = dataset_train["descricao_reclamacao"].apply(lambda x: lemmatize(text=x, mode="all"))

# Vetorização do conjunto de treinamento
vect = vectorize(custom_ngram=1, function="tfid")
vect.fit(dataset_train["descricao_reclamacao_lemmatize_all"])

x_train = vect.transform(dataset_train["descricao_reclamacao_lemmatize_all"])
y_train = dataset_train["categoria"]

In [67]:
# Pré-processamento e remoção de stop words do conjunto de teste
dataset_test["descricao_reclamacao"] = dataset_test["descricao_reclamacao"].apply(lambda x: preprocessing_text(text=x))
dataset_test["descricao_reclamacao"] = dataset_test["descricao_reclamacao"].apply(lambda x: remove_stop_words(text=x, mode="nltk"))
dataset_test["descricao_reclamacao_lemmatize_all"] = dataset_test["descricao_reclamacao"].apply(lambda x: lemmatize(text=x, mode="all"))

# Vetorização do conjunto de teste
x_test = vect.transform(dataset_test["descricao_reclamacao_lemmatize_all"])
y_test = dataset_test["categoria"]

In [68]:
# Treinamento, predição e avaliação dos modelos
experiment  = fit_predict_evaluate_models(x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test)

# Incrementa o contador de experimentos
i += 1

# Adiciona os resultados experimentos ao dicionário de resultados
results[f"{str(i).zfill(2)}"] = experiment

# Apresenta os resultados para cada modelo do experimento
print("DECISION TREE:", results[f"{str(i).zfill(2)}"]["DECISION_TREE"]["weighted avg"])
print("LOGISTIC REGRESSION:", results[f"{str(i).zfill(2)}"]["LOGISTIC_REGRESSION"]["weighted avg"])
print("SUPPORT VECTOR MACHINE:", results[f"{str(i).zfill(2)}"]["SVM"]["weighted avg"])

DECISION TREE: {'precision': 0.7619915545837148, 'recall': 0.7536066818526955, 'f1-score': 0.754989910092383, 'support': 5268.0}
LOGISTIC REGRESSION: {'precision': 0.9099561227691831, 'recall': 0.909832953682612, 'f1-score': 0.909785976636578, 'support': 5268.0}
SUPPORT VECTOR MACHINE: {'precision': 0.9020817743972833, 'recall': 0.9020501138952164, 'f1-score': 0.9019787234789474, 'support': 5268.0}


**Funções aplicadas no experimento 13**

- PREPROCESSING
- REMOVE_STOP_WORDS (SPACY)
- LEMMATIZE (ALL)
- TFID (UNIGRAMA)

In [69]:
# Definição dos conjuntos de treino (75%) e teste (25%)
dataset_train, dataset_test = train_test_split(dataset, train_size=0.75, random_state=42)

In [70]:
# Pré-processamento e remoção de stop words do conjunto de treinamento
dataset_train["descricao_reclamacao"] = dataset_train["descricao_reclamacao"].apply(lambda x: preprocessing_text(text=x))
dataset_train["descricao_reclamacao"] = dataset_train["descricao_reclamacao"].apply(lambda x: remove_stop_words(text=x, mode="spacy"))
dataset_train["descricao_reclamacao_lemmatize_all"] = dataset_train["descricao_reclamacao"].apply(lambda x: lemmatize(text=x, mode="all"))

# Vetorização do conjunto de treinamento
vect = vectorize(custom_ngram=1, function="tfid")
vect.fit(dataset_train["descricao_reclamacao_lemmatize_all"])

x_train = vect.transform(dataset_train["descricao_reclamacao_lemmatize_all"])
y_train = dataset_train["categoria"]

In [71]:
# Pré-processamento e remoção de stop words do conjunto de teste
dataset_test["descricao_reclamacao"] = dataset_test["descricao_reclamacao"].apply(lambda x: preprocessing_text(text=x))
dataset_test["descricao_reclamacao"] = dataset_test["descricao_reclamacao"].apply(lambda x: remove_stop_words(text=x, mode="spacy"))
dataset_test["descricao_reclamacao_lemmatize_all"] = dataset_test["descricao_reclamacao"].apply(lambda x: lemmatize(text=x, mode="all"))

# Vetorização do conjunto de teste
x_test = vect.transform(dataset_test["descricao_reclamacao_lemmatize_all"])
y_test = dataset_test["categoria"]

In [72]:
# Treinamento, predição e avaliação dos modelos
experiment  = fit_predict_evaluate_models(x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test)

# Incrementa o contador de experimentos
i += 1

# Adiciona os resultados experimentos ao dicionário de resultados
results[f"{str(i).zfill(2)}"] = experiment

# Apresenta os resultados para cada modelo do experimento
print("DECISION TREE:", results[f"{str(i).zfill(2)}"]["DECISION_TREE"]["weighted avg"])
print("LOGISTIC REGRESSION:", results[f"{str(i).zfill(2)}"]["LOGISTIC_REGRESSION"]["weighted avg"])
print("SUPPORT VECTOR MACHINE:", results[f"{str(i).zfill(2)}"]["SVM"]["weighted avg"])

DECISION TREE: {'precision': 0.7591868103295649, 'recall': 0.7462034927866363, 'f1-score': 0.7489439636636186, 'support': 5268.0}
LOGISTIC REGRESSION: {'precision': 0.9141233672722235, 'recall': 0.914009111617312, 'f1-score': 0.9139746912547716, 'support': 5268.0}
SUPPORT VECTOR MACHINE: {'precision': 0.9043750532478413, 'recall': 0.9043280182232346, 'f1-score': 0.9042573739009363, 'support': 5268.0}


**Funções aplicadas no experimento 14**

- PREPROCESSING
- REMOVE_STOP_WORDS (NLTK + SPACY)
- LEMMATIZE (ALL)
- TFID (UNIGRAMA)

In [73]:
# Definição dos conjuntos de treino (75%) e teste (25%)
dataset_train, dataset_test = train_test_split(dataset, train_size=0.75, random_state=42)

In [74]:
# Pré-processamento e remoção de stop words do conjunto de treinamento
dataset_train["descricao_reclamacao"] = dataset_train["descricao_reclamacao"].apply(lambda x: preprocessing_text(text=x))
dataset_train["descricao_reclamacao"] = dataset_train["descricao_reclamacao"].apply(lambda x: remove_stop_words(text=x, mode="all"))
dataset_train["descricao_reclamacao_lemmatize_all"] = dataset_train["descricao_reclamacao"].apply(lambda x: lemmatize(text=x, mode="all"))

# Vetorização do conjunto de treinamento
vect = vectorize(custom_ngram=1, function="tfid")
vect.fit(dataset_train["descricao_reclamacao_lemmatize_all"])

x_train = vect.transform(dataset_train["descricao_reclamacao_lemmatize_all"])
y_train = dataset_train["categoria"]

In [75]:
# Pré-processamento e remoção de stop words do conjunto de teste
dataset_test["descricao_reclamacao"] = dataset_test["descricao_reclamacao"].apply(lambda x: preprocessing_text(text=x))
dataset_test["descricao_reclamacao"] = dataset_test["descricao_reclamacao"].apply(lambda x: remove_stop_words(text=x, mode="all"))
dataset_test["descricao_reclamacao_lemmatize_all"] = dataset_test["descricao_reclamacao"].apply(lambda x: lemmatize(text=x, mode="all"))

# Vetorização do conjunto de teste
x_test = vect.transform(dataset_test["descricao_reclamacao_lemmatize_all"])
y_test = dataset_test["categoria"]

In [76]:
# Treinamento, predição e avaliação dos modelos
experiment  = fit_predict_evaluate_models(x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test)

# Incrementa o contador de experimentos
i += 1

# Adiciona os resultados experimentos ao dicionário de resultados
results[f"{str(i).zfill(2)}"] = experiment

# Apresenta os resultados para cada modelo do experimento
print("DECISION TREE:", results[f"{str(i).zfill(2)}"]["DECISION_TREE"]["weighted avg"])
print("LOGISTIC REGRESSION:", results[f"{str(i).zfill(2)}"]["LOGISTIC_REGRESSION"]["weighted avg"])
print("SUPPORT VECTOR MACHINE:", results[f"{str(i).zfill(2)}"]["SVM"]["weighted avg"])

DECISION TREE: {'precision': 0.7559974222862864, 'recall': 0.7443052391799544, 'f1-score': 0.7466467608154437, 'support': 5268.0}
LOGISTIC REGRESSION: {'precision': 0.9137564014762783, 'recall': 0.9136294608959757, 'f1-score': 0.913596097460349, 'support': 5268.0}
SUPPORT VECTOR MACHINE: {'precision': 0.9034597742626403, 'recall': 0.9033788914198937, 'f1-score': 0.9033275555954082, 'support': 5268.0}


**Funções aplicadas no experimento 15**

- PREPROCESSING
- REMOVE_STOP_WORDS (NLTK)
- LEMMATIZE (ALL)
- TFID (BIGRAMA)

In [77]:
# Definição dos conjuntos de treino (75%) e teste (25%)
dataset_train, dataset_test = train_test_split(dataset, train_size=0.75, random_state=42)

In [78]:
# Pré-processamento e remoção de stop words do conjunto de treinamento
dataset_train["descricao_reclamacao"] = dataset_train["descricao_reclamacao"].apply(lambda x: preprocessing_text(text=x))
dataset_train["descricao_reclamacao"] = dataset_train["descricao_reclamacao"].apply(lambda x: remove_stop_words(text=x, mode="nltk"))
dataset_train["descricao_reclamacao_lemmatize_all"] = dataset_train["descricao_reclamacao"].apply(lambda x: lemmatize(text=x, mode="all"))

# Vetorização do conjunto de treinamento
vect = vectorize(custom_ngram=2, function="tfid")
vect.fit(dataset_train["descricao_reclamacao_lemmatize_all"])

x_train = vect.transform(dataset_train["descricao_reclamacao_lemmatize_all"])
y_train = dataset_train["categoria"]

In [79]:
# Pré-processamento e remoção de stop words do conjunto de teste
dataset_test["descricao_reclamacao"] = dataset_test["descricao_reclamacao"].apply(lambda x: preprocessing_text(text=x))
dataset_test["descricao_reclamacao"] = dataset_test["descricao_reclamacao"].apply(lambda x: remove_stop_words(text=x, mode="nltk"))
dataset_test["descricao_reclamacao_lemmatize_all"] = dataset_test["descricao_reclamacao"].apply(lambda x: lemmatize(text=x, mode="all"))

# Vetorização do conjunto de teste
x_test = vect.transform(dataset_test["descricao_reclamacao_lemmatize_all"])
y_test = dataset_test["categoria"]

In [80]:
# Treinamento, predição e avaliação dos modelos
experiment  = fit_predict_evaluate_models(x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test)

# Incrementa o contador de experimentos
i += 1

# Adiciona os resultados experimentos ao dicionário de resultados
results[f"{str(i).zfill(2)}"] = experiment

# Apresenta os resultados para cada modelo do experimento
print("DECISION TREE:", results[f"{str(i).zfill(2)}"]["DECISION_TREE"]["weighted avg"])
print("LOGISTIC REGRESSION:", results[f"{str(i).zfill(2)}"]["LOGISTIC_REGRESSION"]["weighted avg"])
print("SUPPORT VECTOR MACHINE:", results[f"{str(i).zfill(2)}"]["SVM"]["weighted avg"])

DECISION TREE: {'precision': 0.7520347131604599, 'recall': 0.7441154138192863, 'f1-score': 0.7452069215535543, 'support': 5268.0}
LOGISTIC REGRESSION: {'precision': 0.9039861202838666, 'recall': 0.9037585421412301, 'f1-score': 0.9035667005634038, 'support': 5268.0}
SUPPORT VECTOR MACHINE: {'precision': 0.8990904193098319, 'recall': 0.8990129081245254, 'f1-score': 0.8988995684950388, 'support': 5268.0}


**Funções aplicadas no experimento 16**

- PREPROCESSING
- REMOVE_STOP_WORDS (SPACY)
- LEMMATIZE (ALL)
- TFID (BIGRAMA)

In [81]:
# Definição dos conjuntos de treino (75%) e teste (25%)
dataset_train, dataset_test = train_test_split(dataset, train_size=0.75, random_state=42)

In [82]:
# Pré-processamento e remoção de stop words do conjunto de treinamento
dataset_train["descricao_reclamacao"] = dataset_train["descricao_reclamacao"].apply(lambda x: preprocessing_text(text=x))
dataset_train["descricao_reclamacao"] = dataset_train["descricao_reclamacao"].apply(lambda x: remove_stop_words(text=x, mode="spacy"))
dataset_train["descricao_reclamacao_lemmatize_all"] = dataset_train["descricao_reclamacao"].apply(lambda x: lemmatize(text=x, mode="all"))

# Vetorização do conjunto de treinamento
vect = vectorize(custom_ngram=2, function="tfid")
vect.fit(dataset_train["descricao_reclamacao_lemmatize_all"])

x_train = vect.transform(dataset_train["descricao_reclamacao_lemmatize_all"])
y_train = dataset_train["categoria"]

In [83]:
# Pré-processamento e remoção de stop words do conjunto de teste
dataset_test["descricao_reclamacao"] = dataset_test["descricao_reclamacao"].apply(lambda x: preprocessing_text(text=x))
dataset_test["descricao_reclamacao"] = dataset_test["descricao_reclamacao"].apply(lambda x: remove_stop_words(text=x, mode="spacy"))
dataset_test["descricao_reclamacao_lemmatize_all"] = dataset_test["descricao_reclamacao"].apply(lambda x: lemmatize(text=x, mode="all"))

# Vetorização do conjunto de teste
x_test = vect.transform(dataset_test["descricao_reclamacao_lemmatize_all"])
y_test = dataset_test["categoria"]

In [84]:
# Treinamento, predição e avaliação dos modelos
experiment  = fit_predict_evaluate_models(x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test)

# Incrementa o contador de experimentos
i += 1

# Adiciona os resultados experimentos ao dicionário de resultados
results[f"{str(i).zfill(2)}"] = experiment

# Apresenta os resultados para cada modelo do experimento
print("DECISION TREE:", results[f"{str(i).zfill(2)}"]["DECISION_TREE"]["weighted avg"])
print("LOGISTIC REGRESSION:", results[f"{str(i).zfill(2)}"]["LOGISTIC_REGRESSION"]["weighted avg"])
print("SUPPORT VECTOR MACHINE:", results[f"{str(i).zfill(2)}"]["SVM"]["weighted avg"])

DECISION TREE: {'precision': 0.7584931015913955, 'recall': 0.7460136674259681, 'f1-score': 0.7484446372048715, 'support': 5268.0}
LOGISTIC REGRESSION: {'precision': 0.9058310364257547, 'recall': 0.9056567957479119, 'f1-score': 0.9054387880693512, 'support': 5268.0}
SUPPORT VECTOR MACHINE: {'precision': 0.8999936930515755, 'recall': 0.8999620349278664, 'f1-score': 0.8998154848339923, 'support': 5268.0}


**Funções aplicadas no experimento 17**

- PREPROCESSING
- REMOVE_STOP_WORDS (NLTK + SPACY)
- LEMMATIZE (ALL)
- TF (BIGRAMA)

In [85]:
# Definição dos conjuntos de treino (75%) e teste (25%)
dataset_train, dataset_test = train_test_split(dataset, train_size=0.75, random_state=42)

In [86]:
# Pré-processamento e remoção de stop words do conjunto de treinamento
dataset_train["descricao_reclamacao"] = dataset_train["descricao_reclamacao"].apply(lambda x: preprocessing_text(text=x))
dataset_train["descricao_reclamacao"] = dataset_train["descricao_reclamacao"].apply(lambda x: remove_stop_words(text=x, mode="all"))
dataset_train["descricao_reclamacao_lemmatize_all"] = dataset_train["descricao_reclamacao"].apply(lambda x: lemmatize(text=x, mode="all"))

# Vetorização do conjunto de treinamento
vect = vectorize(custom_ngram=2, function="tfid")
vect.fit(dataset_train["descricao_reclamacao_lemmatize_all"])

x_train = vect.transform(dataset_train["descricao_reclamacao_lemmatize_all"])
y_train = dataset_train["categoria"]

In [87]:
# Pré-processamento e remoção de stop words do conjunto de teste
dataset_test["descricao_reclamacao"] = dataset_test["descricao_reclamacao"].apply(lambda x: preprocessing_text(text=x))
dataset_test["descricao_reclamacao"] = dataset_test["descricao_reclamacao"].apply(lambda x: remove_stop_words(text=x, mode="all"))
dataset_test["descricao_reclamacao_lemmatize_all"] = dataset_test["descricao_reclamacao"].apply(lambda x: lemmatize(text=x, mode="all"))

# Vetorização do conjunto de teste
x_test = vect.transform(dataset_test["descricao_reclamacao_lemmatize_all"])
y_test = dataset_test["categoria"]

In [88]:
# Treinamento, predição e avaliação dos modelos
experiment  = fit_predict_evaluate_models(x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test)

# Incrementa o contador de experimentos
i += 1

# Adiciona os resultados experimentos ao dicionário de resultados
results[f"{str(i).zfill(2)}"] = experiment

# Apresenta os resultados para cada modelo do experimento
print("DECISION TREE:", results[f"{str(i).zfill(2)}"]["DECISION_TREE"]["weighted avg"])
print("LOGISTIC REGRESSION:", results[f"{str(i).zfill(2)}"]["LOGISTIC_REGRESSION"]["weighted avg"])
print("SUPPORT VECTOR MACHINE:", results[f"{str(i).zfill(2)}"]["SVM"]["weighted avg"])

DECISION TREE: {'precision': 0.7556247566385215, 'recall': 0.7465831435079726, 'f1-score': 0.7480489244338642, 'support': 5268.0}
LOGISTIC REGRESSION: {'precision': 0.9055919370416847, 'recall': 0.9054669703872438, 'f1-score': 0.9052447891046912, 'support': 5268.0}
SUPPORT VECTOR MACHINE: {'precision': 0.9005444440093939, 'recall': 0.9005315110098709, 'f1-score': 0.9003900101216744, 'support': 5268.0}


**Funções aplicadas no experimento 18**

- START_CLIENT 
- CLASSIFY_TEST (GPT-4, GPT-4O-MINI, GPT-3.5-TURBO e SABIA-3)

In [89]:
# Definição das chaves de autenticação e URL do modelo
API_KEY_GPT: str = os.getenv("GPT_ACCESS_KEY")
API_KEY_MARITACA: str = os.getenv("MARITACA_ACCESS_KEY")
BASE_URL: str = "https://chat.maritaca.ai/api" 

# Inicialização dos clients GPT e MARITACA
client_gpt = start_client(model="gpt-4o-mini", api_key=API_KEY_GPT)
client_maritaca = start_client(model="sabia-3", api_key=API_KEY_MARITACA, base_url=BASE_URL)

# Frases a serem classificadas com o uso dos LLMs
texts : dict = {
    "Cartão de crédito / Cartão pré-pago" : "Atualizei meu cartão xxxx xxxx em xx/xx/2018 e fui informado pelo agente que fez a atualização que minha data de aniversário não mudaria. Ele virou o agente me dando as informações erradas para atualizar a conta. Xxxx alterou minha data de aniversário de xx/xx/xxxx para xx/xx/xxxx sem meu consentimento! XXXX tem a gravação do agente que me enganou.",
    "Hipotecas / Empréstimos" : "Bom dia, meu nome é xxxx xxxx e agradeço se você puder me ajudar a acabar com os serviços de membro do cartão bancário. Em 2018, escrevi para Chase solicitar verificação da dívida e o que eles me enviaram uma declaração que não é aceitável. Estou pedindo ao banco que valide a dívida. Em vez disso, recebi e -mails todos os meses, tentando coletar uma dívida. Tenho o direito de conhecer essas informações como consumidor. Conta do Chase # xxxx xxxx xxxx xxxx Obrigado antecipadamente pela sua ajuda.",
    "Roubo / Relatório de disputa" : "Xxxx xxxx um sofá, assento de amor, mesa e cadeiras e nunca foi entregue. Cartão de débito cobrado {$ 2400.00}. O Banco diz que o Visa negou reivindicação, não temos móveis nem dinheiro.",
    "Serviços de conta bancária" : "Abri a conta de poupança para o bônus {$ 25,00}. Eu deveria receber o bônus {$ 25,00} após três transferências consecutivas de automóveis da verificação para a economia. Percebo em xx/xx/2019 que a transferência automática foi cancelada por fundos insuficientes na minha conta da minha verificação. Portanto, coloquei fundos suficientes em minha conta no XX/XX/2019 solicitou que a equipe executiva reativasse minha transferência automática para o mês de XXXX. Embora a sra. XXXX tenha me procurado do escritório executivo, ela não tentou resolver minhas preocupações (caso # xxxx).",
    "Outros" : "Fiz uma compra de {$ 260,00} em xx/xx/xxxx. Fiz pagamentos de {$ 160,00} em xx/xx/xxxx e {$ 260,00} em xx/xx/xxxx. O pagamento mínimo foi {$ 140,00}. A data de fechamento era xx/xx/xxxx (consulte a instrução anexada). Essa instrução (xx/xx/xxxx) mostra que eu tinha um excelente saldo de compra de {$ 230,00} em xx/xx/xxxx. Eu incluí a declaração dos meses anteriores para mostrar que não havia outras compras anteriores à de XX/XX/XXXX."
}

# Classificação das frases supracitadas com o uso dos LLMs (GPT-4O, GPT-4O-MINI, GPT-3.5-TURBO e SABIA-3)
for key, value in texts.items():
    answer_gpt_4: str = str(classify_text(client=client_gpt, model="gpt-4o", text=value))
    answer_gpt_4_o_mini: str = str(classify_text(client=client_gpt, model="gpt-4o-mini", text=value))
    answer_gpt_3_5_turbo: str = str(classify_text(client=client_gpt, model="gpt-3.5-turbo", text=value))
    answer_maritaca: str = str(classify_text(client=client_maritaca, model="sabia-3", text=value))
    print(f"Categoria esperada: {key}\nGPT-4O: {answer_gpt_4}\nGPT-4O-MINI: {answer_gpt_4_o_mini}\nGPT-3.5-TURBO: {answer_gpt_3_5_turbo}\nSABIA-3: {answer_maritaca}\n")

Categoria esperada: Cartão de crédito / Cartão pré-pago
GPT-4O: {"texto": "Atualizei meu cartão xxxx xxxx em xx/xx/2018 e fui informado pelo agente que fez a atualização que minha data de aniversário não mudaria. Ele virou o agente me dando as informações erradas para atualizar a conta. Xxxx alterou minha data de aniversário de xx/xx/xxxx para xx/xx/xxxx sem meu consentimento! XXXX tem a gravação do agente que me enganou.", "categoria": "cartão de crédito ou cartão pré-pago"}
GPT-4O-MINI: {"texto": "Atualizei meu cartão xxxx xxxx em xx/xx/2018 e fui informado pelo agente que fez a atualização que minha data de aniversário não mudaria. Ele virou o agente me dando as informações erradas para atualizar a conta. Xxxx alterou minha data de aniversário de xx/xx/xxxx para xx/xx/xxxx sem meu consentimento! XXXX tem a gravação do agente que me enganou.", "categoria": "cartão de crédito ou cartão pré-pago"}
GPT-3.5-TURBO: {"texto": "Atualizei meu cartão xxxx xxxx em xx/xx/2018 e fui informado pe

___
#### Conclusões Finais

**F1-score do modelo campeão**

In [90]:
# Inicializa o dicionário para armazenar o F1-score dos modelos
f1: dict = dict()

# Percorre os experimentos realizados
for key in results:
    # Armazena o i-ésimo F1-score dos modelos DecisionTree, LogisticRegression e SVM
    current_decision_tree_f1: float = round(float(results[key]["DECISION_TREE"]["weighted avg"]["f1-score"]) * 100, 2)
    current_logistic_regression_f1: float = round(float(results[key]["LOGISTIC_REGRESSION"]["weighted avg"]["f1-score"]) * 100, 2)
    current_svm_f1: float = round(float(results[key]["SVM"]["weighted avg"]["f1-score"]) * 100, 2)

    # F1-score do DecisionTree superior ao LogisticRegression e SVM
    if current_decision_tree_f1 > current_logistic_regression_f1 and current_decision_tree_f1 > current_svm_f1:
        f1[f"{key} DECISION_TREE"] = current_decision_tree_f1
    
    # F1-score do LogisticRegression superior ao DecisionTree e SVM
    if current_logistic_regression_f1 > current_decision_tree_f1 and current_logistic_regression_f1 > current_svm_f1:
        f1[f"{key} LOGISTIC_REGRESSION"] = current_logistic_regression_f1
    
    # F1-score do SVM superior ao DecisionTree e LogisticRegression
    if current_svm_f1 > current_decision_tree_f1 and current_svm_f1 > current_logistic_regression_f1:
        f1[f"{key} SVM"] = current_svm_f1

# F1-score máximo após avaliação dos modelos
max_f1: float = float(max(f1.values()))

# Definição do modelo campeão
champion: str = str([key for key, value in f1.items() if float(value) == max_f1][0])
print(f"Modelo campeão: {champion}")
print(f"-> F1-Score: {f1.get(champion)}")

Modelo campeão: 13 LOGISTIC_REGRESSION
-> F1-Score: 91.4


**Funções aplicadas no modelo campeão**

- PREPROCESSING
- REMOVE_STOP_WORDS (SPACY)
- LEMMATIZE (ALL)
- TFID (UNIGRAMA)

In [91]:
# Definição dos conjuntos de treino (75%) e teste (25%)
dataset_train, dataset_test = train_test_split(dataset, train_size=0.75, random_state=42)

In [92]:
# Pré-processamento e remoção de stop words do conjunto de treinamento
dataset_train["descricao_reclamacao"] = dataset_train["descricao_reclamacao"].apply(lambda x: preprocessing_text(text=x))
dataset_train["descricao_reclamacao"] = dataset_train["descricao_reclamacao"].apply(lambda x: remove_stop_words(text=x, mode="spacy"))
dataset_train["descricao_reclamacao_lemmatize_all"] = dataset_train["descricao_reclamacao"].apply(lambda x: lemmatize(text=x, mode="all"))

# Vetorização do conjunto de treinamento
vect = vectorize(custom_ngram=1, function="tfid")
vect.fit(dataset_train["descricao_reclamacao_lemmatize_all"])

x_train = vect.transform(dataset_train["descricao_reclamacao_lemmatize_all"])
y_train = dataset_train["categoria"]

In [93]:
# Pré-processamento e remoção de stop words do conjunto de teste
dataset_test["descricao_reclamacao"] = dataset_test["descricao_reclamacao"].apply(lambda x: preprocessing_text(text=x))
dataset_test["descricao_reclamacao"] = dataset_test["descricao_reclamacao"].apply(lambda x: remove_stop_words(text=x, mode="spacy"))
dataset_test["descricao_reclamacao_lemmatize_all"] = dataset_test["descricao_reclamacao"].apply(lambda x: lemmatize(text=x, mode="all"))

# Vetorização do conjunto de teste
x_test = vect.transform(dataset_test["descricao_reclamacao_lemmatize_all"])
y_test = dataset_test["categoria"]

In [94]:
# Treinamento, predição e avaliação dos modelos
experiment  = fit_predict_evaluate_models(x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test)

# Adiciona os resultados experimentos ao dicionário de resultados
results["13"] = experiment

# Apresenta os resultados para cada modelo do experimento
print("DECISION TREE:", results["13"]["DECISION_TREE"]["weighted avg"])
print("LOGISTIC REGRESSION:", results["13"]["LOGISTIC_REGRESSION"]["weighted avg"])
print("SUPPORT VECTOR MACHINE:", results["13"]["SVM"]["weighted avg"])

DECISION TREE: {'precision': 0.7591868103295649, 'recall': 0.7462034927866363, 'f1-score': 0.7489439636636186, 'support': 5268.0}
LOGISTIC REGRESSION: {'precision': 0.9141233672722235, 'recall': 0.914009111617312, 'f1-score': 0.9139746912547716, 'support': 5268.0}
SUPPORT VECTOR MACHINE: {'precision': 0.9043750532478413, 'recall': 0.9043280182232346, 'f1-score': 0.9042573739009363, 'support': 5268.0}


Conforme apresentado acima, o modelo campeão derivou do 13° experimento utilizando a técnica de regressão logística. No experimento, utilizamos a função de pré-processamento, remoção de stop words do SPACY, lematização de todas as palavras e vetorização TFID e obtivemos um F1-score de aproximadamente 91.4%.

Sobre a aplicação dos modelos generativos, entendemos que há oportunidades para explorar outras abordagens, como por exemplo, a utilização de técnicas como o fine tuning para especializar o modelo em um determinado contexto. Entretanto, devido ao custo dos experimentos, optamos neste momento por seguir com uma visão mais simples da aplicação, onde classificamos apenas 5 chamados.